<a href="https://colab.research.google.com/github/liangkang90/machineLearningExercise/blob/master/paddlepaddle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install paddlepaddle

    100% |████████████████████████████████| 49.7MB 927kB/s 
    100% |████████████████████████████████| 348kB 18.0MB/s 
    100% |████████████████████████████████| 901kB 10.1MB/s 
    100% |████████████████████████████████| 112kB 31.2MB/s 
    100% |████████████████████████████████| 512kB 13.6MB/s 
    100% |████████████████████████████████| 12.6MB 3.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/dc/84/da/8aff50941f548db5384b076d5a6a6afea0cd12672e0326edc4
Successfully built rarfile
tweepy 3.6.0 has requirement requests>=2.11.1, but you'll have requests 2.9.2 which is incompatible.
tfds-nightly 1.0.1.dev201902190105 has requirement protobuf>=3.6.1, but you'll have protobuf 3.1.0 which is incompatible.
tensorflow 1.13.0rc2 has requirement protobuf>=3.6.1, but you'll have protobuf 3.1.0 which is incompatible.
tensorflow-metadata 0.9.0 has requirement protobuf<4,>=3.6.0, but you'll have protobuf 3.1.0 which is incompatible.
tensorflow-hub 0.2.0 has requirement protobuf>=3.4.0, but y

In [0]:
#   Copyright (c) 2018 PaddlePaddle Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import print_function

import os
from PIL import Image
import numpy
import paddle
import paddle.fluid as fluid

BATCH_SIZE = 64
PASS_NUM = 5


def loss_net(hidden, label):
    prediction = fluid.layers.fc(input=hidden, size=10, act='softmax')
    loss = fluid.layers.cross_entropy(input=prediction, label=label)
    avg_loss = fluid.layers.mean(loss)
    acc = fluid.layers.accuracy(input=prediction, label=label)
    return prediction, avg_loss, acc


def multilayer_perceptron(img, label):
    img = fluid.layers.fc(input=img, size=200, act='tanh')
    hidden = fluid.layers.fc(input=img, size=200, act='tanh')
    return loss_net(hidden, label)


def softmax_regression(img, label):
    return loss_net(img, label)


def convolutional_neural_network(img, label):
    conv_pool_1 = fluid.nets.simple_img_conv_pool(
        input=img,
        filter_size=5,
        num_filters=20,
        pool_size=2,
        pool_stride=2,
        act="relu")
    conv_pool_1 = fluid.layers.batch_norm(conv_pool_1)
    conv_pool_2 = fluid.nets.simple_img_conv_pool(
        input=conv_pool_1,
        filter_size=5,
        num_filters=50,
        pool_size=2,
        pool_stride=2,
        act="relu")
    return loss_net(conv_pool_2, label)


def train(nn_type,
          use_cuda,
          save_dirname=None,
          model_filename=None,
          params_filename=None):
    if use_cuda and not fluid.core.is_compiled_with_cuda():
        return

    img = fluid.layers.data(name='img', shape=[1, 28, 28], dtype='float32')
    label = fluid.layers.data(name='label', shape=[1], dtype='int64')

    if nn_type == 'softmax_regression':
        net_conf = softmax_regression
    elif nn_type == 'multilayer_perceptron':
        net_conf = multilayer_perceptron
    else:
        net_conf = convolutional_neural_network

    prediction, avg_loss, acc = net_conf(img, label)

    test_program = fluid.default_main_program().clone(for_test=True)

    optimizer = fluid.optimizer.Adam(learning_rate=0.001)
    optimizer.minimize(avg_loss)

    def train_test(train_test_program, train_test_feed, train_test_reader):
        acc_set = []
        avg_loss_set = []
        for test_data in train_test_reader():
            acc_np, avg_loss_np = exe.run(
                program=train_test_program,
                feed=train_test_feed.feed(test_data),
                fetch_list=[acc, avg_loss])
            acc_set.append(float(acc_np))
            avg_loss_set.append(float(avg_loss_np))
        # get test acc and loss
        acc_val_mean = numpy.array(acc_set).mean()
        avg_loss_val_mean = numpy.array(avg_loss_set).mean()
        return avg_loss_val_mean, acc_val_mean

    place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()

    exe = fluid.Executor(place)

    train_reader = paddle.batch(
        paddle.reader.shuffle(paddle.dataset.mnist.train(), buf_size=500),
        batch_size=BATCH_SIZE)
    test_reader = paddle.batch(
        paddle.dataset.mnist.test(), batch_size=BATCH_SIZE)
    feeder = fluid.DataFeeder(feed_list=[img, label], place=place)

    exe.run(fluid.default_startup_program())
    main_program = fluid.default_main_program()
    epochs = [epoch_id for epoch_id in range(PASS_NUM)]

    lists = []
    step = 0
    for epoch_id in epochs:
        for step_id, data in enumerate(train_reader()):
            metrics = exe.run(
                main_program,
                feed=feeder.feed(data),
                fetch_list=[avg_loss, acc])
            if step % 100 == 0:
                print("Pass %d, Batch %d, Cost %f" % (step, epoch_id,
                                                      metrics[0]))
            step += 1
        # test for epoch
        avg_loss_val, acc_val = train_test(
            train_test_program=test_program,
            train_test_reader=test_reader,
            train_test_feed=feeder)

        print("Test with Epoch %d, avg_cost: %s, acc: %s" %
              (epoch_id, avg_loss_val, acc_val))
        lists.append((epoch_id, avg_loss_val, acc_val))
        if save_dirname is not None:
            fluid.io.save_inference_model(
                save_dirname, ["img"], [prediction],
                exe,
                model_filename=model_filename,
                params_filename=params_filename)

    # find the best pass
    best = sorted(lists, key=lambda list: float(list[1]))[0]
    print('Best pass is %s, testing Avgcost is %s' % (best[0], best[1]))
    print('The classification accuracy is %.2f%%' % (float(best[2]) * 100))



In [0]:
def infer(use_cuda,
          save_dirname=None,
          model_filename=None,
          params_filename=None):
    if save_dirname is None:
        return

    place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
    exe = fluid.Executor(place)

    def load_image(file):
        im = Image.open(file).convert('L')
        im = im.resize((28, 28), Image.ANTIALIAS)
        im = numpy.array(im).reshape(1, 1, 28, 28).astype(numpy.float32)
        im = im / 255.0 * 2.0 - 1.0
        return im

  #  cur_dir = os.path.dirname(os.path.realpath(__file__))
  #  tensor_img = load_image(cur_dir + '/image/infer_3.png')
    tensor_img = load_image('/content/gdrive/My Drive/image/photo1.png')

    inference_scope = fluid.core.Scope()
    with fluid.scope_guard(inference_scope):
        # Use fluid.io.load_inference_model to obtain the inference program desc,
        # the feed_target_names (the names of variables that will be feeded
        # data using feed operators), and the fetch_targets (variables that
        # we want to obtain data from using fetch operators).
        [inference_program, feed_target_names,
         fetch_targets] = fluid.io.load_inference_model(
             save_dirname, exe, model_filename, params_filename)

        # Construct feed as a dictionary of {feed_target_name: feed_target_data}
        # and results will contain a list of data corresponding to fetch_targets.
        results = exe.run(
            inference_program,
            feed={feed_target_names[0]: tensor_img},
            fetch_list=fetch_targets)
        lab = numpy.argsort(results)
        print("Inference result of image/infer_3.png is: %d" % lab[0][0][-1])


def main(use_cuda, nn_type):
    model_filename = None
    params_filename = None
    save_dirname = "recognize_digits_" + nn_type + ".inference.model"

    # call train() with is_local argument to run distributed train
    train(
        nn_type=nn_type,
        use_cuda=use_cuda,
        save_dirname=save_dirname,
        model_filename=model_filename,
        params_filename=params_filename)
  


In [0]:
if __name__ == '__main__':
    use_cuda = False
    # predict = 'softmax_regression' # uncomment for Softmax
    # predict = 'multilayer_perceptron' # uncomment for MLP
    predict = 'convolutional_neural_network'  # uncomment for LeNet5
 

In [0]:
   main(use_cuda=use_cuda, nn_type=predict)

[==================================================]
[==================================================]
[==================================================]
[==================================================]


Pass 0, Batch 0, Cost 3.483964
Pass 100, Batch 0, Cost 0.215237
Pass 200, Batch 0, Cost 0.258138
Pass 300, Batch 0, Cost 0.206161
Pass 400, Batch 0, Cost 0.155898
Pass 500, Batch 0, Cost 0.116960
Pass 600, Batch 0, Cost 0.076878
Pass 700, Batch 0, Cost 0.064189
Pass 800, Batch 0, Cost 0.031875
Pass 900, Batch 0, Cost 0.044066
Test with Epoch 0, avg_cost: 0.09022811680897228, acc: 0.9709394904458599
Pass 1000, Batch 1, Cost 0.031239
Pass 1100, Batch 1, Cost 0.039900
Pass 1200, Batch 1, Cost 0.046206
Pass 1300, Batch 1, Cost 0.029625
Pass 1400, Batch 1, Cost 0.048268
Pass 1500, Batch 1, Cost 0.040052
Pass 1600, Batch 1, Cost 0.016050
Pass 1700, Batch 1, Cost 0.007789
Pass 1800, Batch 1, Cost 0.024415
Test with Epoch 1, avg_cost: 0.06500062548484435, acc: 0.9793988853503185
Pass 1900, Batch 2, Cost 0.007077
Pass 2000, Batch 2, Cost 0.111271
Pass 2100, Batch 2, Cost 0.078238
Pass 2200, Batch 2, Cost 0.005146
Pass 2300, Batch 2, Cost 0.021095
Pass 2400, Batch 2, Cost 0.012375
Pass 2500, Bat

FileNotFoundError: ignored